In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install sentencepiece accelerate einops
! pip install codecarbon

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_zedx89c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_zedx89c
  Resolved https://github.com/huggingface/transformers to commit 706703bba6c920b10aa7e7ee8163b06a8a03c450
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10960478 sha256=c9cfc6fb1e2c44359bc0ff992773773365c659f113a02eb029cdd5f3204011c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-gqw9grzq/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
  Using cached nvidia_cuda_nvrtc_cu12-1

In [2]:
from codecarbon import EmissionsTracker
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
from huggingface_hub import snapshot_download

In [5]:
repo_id = 'microsoft/phi-2'

model_path = snapshot_download(repo_id=repo_id,repo_type="model", local_dir="./phi-2", local_dir_use_symlinks=False)

model_path = "./phi-2"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:832: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

NOTICE.md:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

SECURITY.md:   0%|          | 0.00/2.66k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Needs 12GB of vRAM to run in float32 (default)
# Run this line to load in float16. You need Gb of vRAM
torch.set_default_dtype(torch.float16)

model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
model.device

device(type='cuda', index=0)

## Original Generate Function without being optimized

In [10]:
import time
def generate(prompt: str, generation_params: dict = {"max_length":200})-> str :

    #codecarbon stuff
    tracker = EmissionsTracker()
    tracker.start()

    s = time.time()

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, **generation_params)
    completion = tokenizer.batch_decode(outputs)[0]

    elapsed = time.time() - s

    num_input_tokens = inputs['input_ids'].shape[1]
    num_total_tokens = outputs.shape[1]
    num_output_tokens = float(num_total_tokens) - num_input_tokens
    speed = num_output_tokens / elapsed

    #codecarbon stuff
    emissions: float = tracker.stop()
    print(f"\n Emissions: {emissions*1000:.3f} grams \n")
    print(f"Took {round(elapsed,1)} seconds to generate {int(num_output_tokens)} new tokens at speed {round(speed, 1)} tokens/seconds")

    return completion

In [11]:
prompt = """
Create a list of 5 words which have a similar meaning to the word hope.

"""

## Energy output of the original unoptimized code

In [12]:
result = generate(prompt, generation_params={"max_length":200})
result

[codecarbon INFO @ 06:37:21] [setup] RAM Tracking...
[codecarbon INFO @ 06:37:21] [setup] CPU Tracking...
[codecarbon WARNING @ 06:37:21] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 06:37:23] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 06:37:23] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 06:37:23] [setup] GPU Tracking...
[codecarbon INFO @ 06:37:23] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 06:37:23] >>> Tracker's metadata:
[codecarbon INFO @ 06:37:23]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 06:37:23]   Python version: 3.11.11
[codecarbon INFO @ 06:37:23]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 06:37:23]   Available RAM : 12.675 GB
[codecarbon INFO @ 06:37:23]   CPU count: 2
[codeca


 Emissions: 0.013 grams 

Took 3.3 seconds to generate 63 new tokens at speed 19.0 tokens/seconds


/usr/local/lib/python3.11/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


"\nCreate a list of 5 words which have a similar meaning to the word hope.\n\n\n\nSolution:\n\n```python\n# Creating a list of words with similar meaning to hope\nwords = ['optimism', 'faith', 'aspiration', 'desire', 'expectation']\n\n# Printing the list of words\nprint(words)\n```\n\n\n<|endoftext|>"

## JIT Optimization of model

In [13]:
if torch.__version__ >= "2.0":
    model = torch.compile(model)  # JIT optimization for PyTorch 2.0+

model.half()

OptimizedModule(
  (_orig_mod): PhiForCausalLM(
    (model): PhiModel(
      (embed_tokens): Embedding(51200, 2560)
      (layers): ModuleList(
        (0-31): 32 x PhiDecoderLayer(
          (self_attn): PhiAttention(
            (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (dense): Linear(in_features=2560, out_features=2560, bias=True)
          )
          (mlp): PhiMLP(
            (activation_fn): NewGELUActivation()
            (fc1): Linear(in_features=2560, out_features=10240, bias=True)
            (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          )
          (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (rotary_emb): PhiRotaryEmbedding()
      (embed_dropout): Dro

## Generate Function Optimized for Energy Efficiency

In [14]:
def generate(prompt: str, generation_params: dict = {"max_length":200})-> str:
    """Generate text using the Phi-2 model with optimized settings."""

    with torch.no_grad():  # Reduce memory usage
        tracker = EmissionsTracker()
        tracker.start()
        start_time = time.time()

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, **generation_params)
        completion = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        elapsed = time.time() - start_time

        num_input_tokens = inputs['input_ids'].shape[1]
        num_total_tokens = outputs.shape[1]
        num_output_tokens = num_total_tokens - num_input_tokens
        speed = num_output_tokens / elapsed

        print(f"Took {round(elapsed, 1)} sec to generate {int(num_output_tokens)} tokens at {round(speed, 1)} tokens/sec")
        emissions: float = tracker.stop()
        print(f"\n Emissions: {emissions*1000:.3f} grams \n")
        return completion

# Free up GPU memory
torch.cuda.empty_cache()

In [15]:
result = generate(prompt, generation_params={"max_length":200})
result

[codecarbon INFO @ 06:38:12] [setup] RAM Tracking...
[codecarbon INFO @ 06:38:12] [setup] CPU Tracking...
[codecarbon WARNING @ 06:38:12] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 06:38:13] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 06:38:13] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 06:38:13] [setup] GPU Tracking...
[codecarbon INFO @ 06:38:13] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 06:38:13] >>> Tracker's metadata:
[codecarbon INFO @ 06:38:13]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 06:38:13]   Python version: 3.11.11
[codecarbon INFO @ 06:38:13]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 06:38:13]   Available RAM : 12.675 GB
[codecarbon INFO @ 06:38:13]   CPU count: 2
[codeca

Took 2.0 sec to generate 63 tokens at 31.0 tokens/sec

 Emissions: 0.009 grams 



"\nCreate a list of 5 words which have a similar meaning to the word hope.\n\n\n\nSolution:\n\n```python\n# Creating a list of words with similar meaning to hope\nwords = ['optimism', 'faith', 'aspiration', 'desire', 'expectation']\n\n# Printing the list of words\nprint(words)\n```\n\n\n"